In [1]:
from pip_flow.pip_flow import PipFlow


pip_flow=PipFlow(device='cloud', model_key="PipableAI/pipable-worflow-automaton-private-v2")

In [ ]:
prompt="""
<question>
write a python function to double the input number.
</question>
"""
print(pip_flow.generate(prompt, eos_token="code", max_new_tokens=50))

In [2]:
import requests
from bs4 import BeautifulSoup

functions = [
requests.get,
BeautifulSoup,
BeautifulSoup.find_all,
]
# docs= pip_flow.generate_docs(requests.post)
# print(docs)
pip_flow.register_callables(functions,generate_docs=False) #keeps on appending to the object

In [ ]:
pip_flow.functions

In [ ]:
prompt = """
<functions>
{func_info}
</functions>
<json_structure>
{{
  "tasks": [
    {{
      "task_id": 1,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }},
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }}
      ],
      "outputs": ["variable_1"],
      "description": "some description"
    }},
    {{
      "task_id": 2,
      "function_name": "function name",
      "parameters": [
        {{
        "name":"self",
        "value":"variable name to be passed for this parameter self.",
        "dtype":"type annotation of the self parameter",
        "description": "An explanation of why the cariable should be used for this self parameter."
        }},
        {{,
        "name":"name of this parameter according to annotations.",
        "value":"value to be passed for this parameter",
        "dtype":"type annotation of the variable",
        "description": "An explanation of why this value should be utilized."
        }}
      ],
      "outputs": ["variable_2"],
      "description": "some description"
    }}
  ]
}}
</json_structure>
<instructions>
- use self parameter with proper value based on the question.
- name outputs as variable_1 , variable_2 , variable_3 , variable_4 and more variables in chronological order.
- give attention to the type annotation of the parameter given while filling values.
{instructions}
</instructions>
<question>
Given the above functions,
- Do not give the parameters in json which have null values and default values of the function, only give the sequencial function calls with parameters to execute the below question:
{question}
</question>
"""
pip_flow.add_plan_template("new2", prompt)

In [ ]:
pip_flow.load_templates()
pip_flow.prompt_templates

In [ ]:

config = {
"func_info":  str([
                f"""--name:{function.name}\n--annotations:{function.signature}\n--doc:{function.docs}\n\n"""
                for function in pip_flow.functions
            ]),
"instructions": ""
}
print(pip_flow.make_live_prompt("new", config))

In [ ]:
print(pip_flow.last_base_prompt)
pip_flow.latest_config

In [3]:
ques = "get the content of https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw create beautiful soup class and find all divs in it"
plan =pip_flow.generate_plan(ques)
print(plan)

{
    "tasks": [
        {
            "task_id": 1,
            "function_name": "get",
            "parameters": [
                {
                    "name": "url",
                    "value": "https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw",
                    "description": "URL of the page to be scraped.",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_1"
            ],
            "description": "Fetches the content of the webpage."
        },
        {
            "task_id": 2,
            "function_name": "BeautifulSoup",
            "parameters": [
                {
                    "name": "content",
                    "value": "variable_1",
                    "description": "Content of the webpage fetched.",
                    "dtype": "string"
                }
            ],
            "outputs": [
                "variable_2"
 

In [4]:
pip_flow.plan_to_code()


```python
import requests
from bs4 import BeautifulSoup

# Fetch the content of the webpage
url = 'https://colab.research.google.com/drive/1mB2v-d72k5DJ2sXqm7GsaAZdWOJBmmof?authuser=1#scrollTo=9NgzJt0-FoNw'
response = requests.api.get(url)
content = response.text

# Parse the webpage content into a BeautifulSoup object
soup = BeautifulSoup(content, 'html.parser')

# Find all div elements in the BeautifulSoup object
divs = soup.find_all('div')

# Print the number of div elements
print(len(divs))
```


In [5]:
pip_flow.visualise_plan()

network.html


/Users/avikothari/Pipable/pip-flow/pip_flow/pip_flow.py:273: UserWarning: Visualisation is not supported in VS code yet.
  warnings.warn("Visualisation is not supported in VS code yet.")
